In [5]:
import pandas as pd
import numpy as np

import sys
import os
import re

sys.path.append( "../..")

from global_config import config

results_dir = config.get_property("results_dir")
data_dir    = config.get_property("data_dir")

In [6]:

## INFO FROM Bedford lab
# https://github.com/blab/dengue-antigenic-dynamics/blob/dependabot/pip/titer_model/implementation-nextstrain-augur/cvxopt-1.2.7/titer_model/implementation-nextstrain-augur/dengue/dengue.prepare.py
dropped_strains = [
    'DENV1/VIETNAM/BIDV992/2006',        'DENV1/FRANCE/00475/2008', 'DENV1/VIETNAM/BIDV3990/2008', 'DENV2/HAITI/DENGUEVIRUS2HOMOSAPIENS1/2016', # Probable recombinants
    'DENV2/AUSTRALIA/QML22/2015',        # Suspiciously far diverged
    'DENV2/MALAYSIA/DKD811/2008',        'DENV2/MALAYSIA/P81407/1970', 'DENV2/SENEGAL/0674/1970', 'DENV2/SENEGAL/DAKAR0761/1974',                  # Sylvatic
    'DENV2/NIGERIA/IBH11234/1966',       'DENV2/NIGERIA/IBH11664/1966', 'DENV2/NIGERIA/IBH11208/1966', 'DENV2/SENEGAL/DAKARD75505/1999',          # Sylvatic
    'DENV2/SENEGAL/DAKAR141069/1999',    'DENV2/SENEGAL/DAKAR141070/1999', 'DENV2/GUINEA/PM33974/1981', 'DENV2/BURKINA_FASO/DAKAR2039/1980',   # Sylvatic
    'DENV2/COTE_D_IVOIRE/DAKAR578/1980', 'DENV2/COTE_D_IVOIRE/DAKAR510/1980', 'DENV2/MALAYSIA/SAB/2015', 'DENV2/TRINIDAD_AND_TOBAGO/NA/1953'# Sylvatic
    'DENV4/MALAYSIA/P731120/1973',       'DENV4/MALAYSIA/P215/1975' # Sylvatic
]

sanofi_vaccine_strains = {
    'denv1': 'DENV1/THAILAND/PUO359/1980',
    'denv2': 'DENV2/THAILAND/PUO218/1980',
    'denv3': 'DENV3/THAILAND/PAH88188/1988',
    'denv4': 'DENV4/INDONESIA/S1228/1978'}

references = {
    "denv1": {"metadata": {'strain': "DENV1/NAURUISLAND/REFERENCE/1997", "accession": "NC_001477", "date": "1997-XX-XX", 'host': "NA", 'country': "Nauru", 'region': "oceania"}},
    "denv2": {"metadata": {'strain': "DENV2/THAILAND/REFERENCE/1964", "accession": "NC_001474", "date": "1964-XX-XX", 'host': "NA", 'country': "Thailand", "region": "southeast_asia"}},
    "denv3": {"metadata": {'strain': "DENV3/SRI_LANKA/REFERENCE/2000", "accession": "NC_001475", "date": "2000-XX-XX", 'host': "NA", 'country': "Sri Lanka", "region": "south_asia"}},
    "denv4": {"metadata": {'strain': "DENV4/NA/REFERENCE/2003", "accession": "NC_002640", "date": "2003-XX-XX", 'host': "NA", 'country': "NA", "region": "NA"}},
}

In [7]:
from Bio import SeqIO

fasta_files   = [os.path.join(data_dir, "nextstrain", f"sequences_denv{z}.fasta") for z in range (1, 4+1)]
references_gb = [os.path.join(data_dir, "reference", f"dengue_{z}_outgroup.gb") for z in range (1, 4+1)]

seq_file   = fasta_files[0]
ref_file   = references_gb[0]

references = SeqIO.read(ref_file, 'genbank')
sequences  = {x.name:x for x in SeqIO.parse(seq_file, 'fasta')}

from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq

seqs = {}
for name, data in sequences.items():
    seqs[name] = SeqRecord(Seq(data.seq),
            id=name, name=name, description=name)
    seqs[name].attributes = data.annotations

data = {}
data['sequences'] = seqs
data['reference'] = references
data["serotype"]  = "denv1"

In [21]:
data['sequences']['DENV1/INDIA/237/1962'].seq

Seq('atgcgatgtgtgggaataggcaacagagacttcgttgaaggcctgccaggagca...gcg')

In [8]:
seq_df = pd.DataFrame(columns=["name", "serotype", "sequence"])

for sero_idx in range(1, 4+1):
    f     = open(fasta_file,'r')
    lines = f.readlines()
    hre   = re.compile('>(\S+)')
    lre   = re.compile('^(\S+)$')
    gene  = {}

    for line in lines:
            outh = hre.search(line)
            if outh:
                    id   = outh.group(1)
            else:
                    outl = lre.search(line)
                    if(id in gene.keys()):
                            gene[id] += outl.group(1)
                    else:
                            gene[id] = outl.group(1)

    sero_df             = pd.DataFrame.from_dict(gene, orient='index').reset_index().rename(columns={'index':'name', 0:'sequence'})
    sero_df["serotype"] = f"denv{sero_idx}"
    seq_df              = seq_df.append(sero_df)
seq_df["length"] = seq_df["sequence"].apply(lambda x: len(x))

NameError: name 'fasta_file' is not defined

In [ ]:
from Bio import Align

seq1      = str(data["sequences"][list(data["sequences"].keys())[0]].seq)
ref_seq   = data['reference'].seq

aligner   = Align.PairwiseAligner()
alignment = aligner.align(str(ref_seq), str(seq1).upper(), )

In [ ]:
from Bio.Align import MultipleSeqAlignment
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq

sequences = [SeqRecord(Seq(str(seq.seq).upper())) for _, seq in data["sequences"].items()]
ali       = MultipleSeqAlignment(sequences)

ValueError: Sequences must all be the same length

In [ ]:
list(data["sequences"].keys())[5]

'DENV1/BRAZIL/19RJ/2010'